# Generativne mreže

Rekurentne nevronske mreže (RNN) in njihove različice z vrati, kot so celice Long Short Term Memory (LSTM) in Gated Recurrent Units (GRU), omogočajo modeliranje jezika, tj. lahko se naučijo vrstnega reda besed in napovedujejo naslednjo besedo v zaporedju. To nam omogoča uporabo RNN za **generativne naloge**, kot so običajno generiranje besedila, strojno prevajanje in celo opisovanje slik.

V arhitekturi RNN, ki smo jo obravnavali v prejšnji enoti, je vsaka enota RNN ustvarila naslednje skrito stanje kot izhod. Vendar pa lahko vsaki rekurentni enoti dodamo še en izhod, kar nam omogoča, da ustvarimo **zaporedje** (ki je enako dolžini izvirnega zaporedja). Poleg tega lahko uporabimo RNN enote, ki ne sprejemajo vhoda na vsakem koraku, temveč le začetni vektorski stanji, nato pa ustvarijo zaporedje izhodov.

V tem zvezku se bomo osredotočili na preproste generativne modele, ki nam pomagajo ustvarjati besedilo. Za enostavnost bomo zgradili **mrežo na ravni znakov**, ki generira besedilo črko za črko. Med učenjem moramo vzeti neko besedilno zbirko in jo razdeliti na zaporedja črk.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Gradnja besedišča znakov

Za gradnjo generativne mreže na ravni znakov moramo besedilo razdeliti na posamezne znake namesto besed. To lahko dosežemo z definiranjem drugačnega tokenizatorja:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Poglejmo primer, kako lahko kodiramo besedilo iz našega nabora podatkov:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Učenje generativnega RNN

Način, kako bomo učili RNN za generiranje besedila, je naslednji. Na vsakem koraku bomo vzeli zaporedje znakov dolžine `nchars` in mreži naročili, naj za vsak vhodni znak ustvari naslednji izhodni znak:

![Slika, ki prikazuje primer generiranja besede 'HELLO' z RNN.](../../../../../translated_images/sl/rnn-generate.56c54afb52f9781d.webp)

Odvisno od dejanskega scenarija bomo morda želeli vključiti tudi nekatere posebne znake, kot je *konec zaporedja* `<eos>`. V našem primeru želimo mrežo naučiti generiranja neskončnega besedila, zato bomo določili, da je velikost vsakega zaporedja enaka `nchars` znakom. Posledično bo vsak učni primer sestavljen iz `nchars` vhodov in `nchars` izhodov (kar je vhodno zaporedje, premaknjeno za en simbol v levo). Miniserija bo sestavljena iz več takšnih zaporedij.

Način, kako bomo generirali miniserije, je, da vzamemo vsak novičarski tekst dolžine `l` in iz njega ustvarimo vse možne kombinacije vhod-izhod (takšnih kombinacij bo `l-nchars`). Te bodo tvorile eno miniserijo, velikost miniserij pa bo pri vsakem učnem koraku različna.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Zdaj bomo definirali generatorno mrežo. Lahko temelji na katerikoli ponavljajoči se celici, o katerih smo govorili v prejšnji enoti (preprosta, LSTM ali GRU). V našem primeru bomo uporabili LSTM.

Ker mreža sprejema znake kot vhod, velikost besedišča pa je precej majhna, ne potrebujemo vgradne plasti; enovrstni kodiran vhod lahko neposredno preide v LSTM celico. Vendar pa, ker posredujemo številke znakov kot vhod, jih moramo pred posredovanjem v LSTM enovrstno kodirati. To se izvede z uporabo funkcije `one_hot` med prehodom `forward`. Izhodni kodirnik bo linearna plast, ki bo pretvorila skrito stanje v enovrstno kodiran izhod.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Med usposabljanjem želimo imeti možnost vzorčenja generiranega besedila. Da bi to dosegli, bomo definirali funkcijo `generate`, ki bo ustvarila izhodni niz dolžine `size`, začenši z začetnim nizom `start`.

Postopek deluje na naslednji način. Najprej bomo celoten začetni niz poslali skozi mrežo, kjer bomo dobili izhodno stanje `s` in naslednji napovedani znak `out`. Ker je `out` enovročno kodiran (one-hot encoded), uporabimo `argmax`, da dobimo indeks znaka `nc` v besedišču, nato pa uporabimo `itos`, da ugotovimo dejanski znak in ga dodamo v rezultatni seznam znakov `chars`. Ta postopek generiranja enega znaka ponovimo `size`-krat, da ustvarimo zahtevano število znakov.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Zdaj pa začnimo z usposabljanjem! Zanka za usposabljanje je skoraj enaka kot v vseh naših prejšnjih primerih, vendar namesto natančnosti vsakih 1000 epohov izpišemo vzorčno generirano besedilo.

Posebno pozornost je treba nameniti načinu izračuna izgube. Izgubo moramo izračunati glede na enovrstno kodiran izhod `out` in pričakovano besedilo `text_out`, ki je seznam indeksov znakov. Na srečo funkcija `cross_entropy` pričakuje nenormaliziran izhod mreže kot prvi argument in številko razreda kot drugi argument, kar je točno tisto, kar imamo. Prav tako samodejno izvaja povprečenje glede na velikost minibatcha.

Usposabljanje omejimo tudi na `samples_to_train` vzorce, da ne čakamo predolgo. Spodbujamo vas, da eksperimentirate in poskusite daljše usposabljanje, morda za več epohov (v tem primeru bi morali ustvariti dodatno zanko okoli te kode).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Ta primer že ustvarja precej dober tekst, vendar ga je mogoče izboljšati na več načinov:

* **Boljša priprava minibatch-ov**. Način, kako smo pripravili podatke za učenje, je bil, da smo iz enega vzorca ustvarili en minibatch. To ni idealno, saj so minibatch-i različnih velikosti, nekateri pa sploh ne morejo biti ustvarjeni, ker je besedilo krajše od `nchars`. Poleg tega majhni minibatch-i ne obremenijo GPU dovolj učinkovito. Bolj smiselno bi bilo vzeti en velik kos besedila iz vseh vzorcev, nato ustvariti vse pare vhod-izhod, jih premešati in ustvariti minibatch-e enake velikosti.

* **Večplastni LSTM**. Smiselno je poskusiti z 2 ali 3 plastmi LSTM celic. Kot smo omenili v prejšnji enoti, vsaka plast LSTM izlušči določene vzorce iz besedila, in pri generatorju na ravni znakov lahko pričakujemo, da bo nižja raven LSTM odgovorna za izluščanje zlogov, višje ravni pa za besede in besedne zveze. To je mogoče preprosto implementirati z dodajanjem parametra za število plasti v konstruktor LSTM.

* Prav tako bi morda želeli eksperimentirati z **GRU enotami** in preveriti, katere delujejo bolje, ter z **različnimi velikostmi skritih plasti**. Prevelika skrita plast lahko privede do overfittinga (npr. mreža se bo naučila točno določenega besedila), premajhna velikost pa morda ne bo dala dobrih rezultatov.


## Mehko generiranje besedila in temperatura

V prejšnji definiciji funkcije `generate` smo vedno izbrali znak z najvišjo verjetnostjo kot naslednji znak v generiranem besedilu. To je pogosto povzročilo, da se je besedilo "ponavljalo" med istimi zaporedji znakov znova in znova, kot v tem primeru:
```
today of the second the company and a second the company ...
```

Če pa pogledamo porazdelitev verjetnosti za naslednji znak, lahko opazimo, da razlika med nekaj najvišjimi verjetnostmi ni velika, npr. en znak ima verjetnost 0,2, drugi pa 0,19 itd. Na primer, ko iščemo naslednji znak v zaporedju '*play*', je lahko naslednji znak prav tako presledek ali **e** (kot v besedi *player*).

To nas pripelje do zaključka, da ni vedno "pravično" izbrati znaka z višjo verjetnostjo, saj lahko izbira drugega najverjetnejšega znaka še vedno vodi do smiselnega besedila. Bolj smiselno je **vzeti vzorec** znakov iz porazdelitve verjetnosti, ki jo poda izhod mreže.

To vzorčenje lahko izvedemo z uporabo funkcije `multinomial`, ki implementira tako imenovano **multinomsko porazdelitev**. Funkcija, ki implementira to **mehko** generiranje besedila, je definirana spodaj:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Predstavili smo še en parameter, imenovan **temperatura**, ki se uporablja za označevanje, kako strogo se moramo držati najvišje verjetnosti. Če je temperatura 1,0, izvajamo pošteno multinomialno vzorčenje, in ko temperatura gre proti neskončnosti - vse verjetnosti postanejo enake, naključno izberemo naslednji znak. V spodnjem primeru lahko opazimo, da besedilo postane nesmiselno, ko preveč povečamo temperaturo, in spominja na "ciklirano" težko generirano besedilo, ko se približa 0.



---

**Omejitev odgovornosti**:  
Ta dokument je bil preveden z uporabo storitve za strojno prevajanje [Co-op Translator](https://github.com/Azure/co-op-translator). Čeprav si prizadevamo za natančnost, vas prosimo, da upoštevate, da lahko avtomatizirani prevodi vsebujejo napake ali netočnosti. Izvirni dokument v njegovem izvirnem jeziku je treba obravnavati kot avtoritativni vir. Za ključne informacije priporočamo strokovni človeški prevod. Ne prevzemamo odgovornosti za morebitna nesporazumevanja ali napačne razlage, ki izhajajo iz uporabe tega prevoda.
